In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import yaml
import os, os.path, sys, logging
from functools import reduce
import re, random, scipy.misc
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

## Get Dataset

In [ ]:
# Get the training/test set information
def get_all_labels(input_yaml, riib=False):
    """ Gets all labels within label file
    Note that RGB images are 1280x720 and RIIB images are 1280x736.
    :param input_yaml: Path to yaml file
    :param riib: If True, change path to labeled pictures
    :return: images: Labels for traffic lights
    """
    images = yaml.load(open(input_yaml, 'rb').read())

    for i in range(len(images)):
        images[i]['path'] = os.path.abspath(os.path.join(os.path.dirname(input_yaml), images[i]['path']))
        if riib:
            images[i]['path'] = images[i]['path'].replace('.png', '.pgm')
            images[i]['path'] = images[i]['path'].replace('rgb/train', 'riib/train')
            images[i]['path'] = images[i]['path'].replace('rgb/test', 'riib/test')
            for box in images[i]['boxes']:
                box['y_max'] = box['y_max'] + 8
                box['y_min'] = box['y_min'] + 8
    return images

In [ ]:
## BOSCH DATASET
# train_yaml = "images/train.yaml"
# test_yaml = "images/test.yaml"

## SIMULATOR DATASET
simulator_yaml = "simulator/simulator_final.yaml"
dataset = get_all_labels(simulator_yaml)

# Split into paths and labels
paths = []
labels = []
for data in dataset:
    paths.append(data['path'])
    
    # Determine label
    if len(data['boxes']) == 0:
        labels.append('NoLight')
    else:
        labels.append(data['boxes'][0]['label'])

## Create Train, Validation, and Test Sets

In [ ]:
# Train, validation, and test sets
# Split ratio is 7:3:1
x_train, x_not_train, y_train, y_not_train = train_test_split(paths, labels, test_size=0.3)
x_validation, x_test, y_validation, y_test = train_test_split(x_not_train, y_not_train, test_size=0.33)

print("Train size: {}, Validation size: {}, Test size: {}".format( \
    len(x_train), len(x_validation), len(x_test))) 
print("All labels in training set:", set(y_train))
print("All labels in validation set:", set(y_validation))
print("All labels in test set:", set(y_test))

In [ ]:
# Label counts in training set - Needed for oversampling
label_counts = {}
label_indicies = {}
for i, label in enumerate(y_train):
    if label not in label_counts:
        label_counts[label] = 0
        label_indicies[label] = []
    label_counts[label] += 1
    label_indicies[label].append(i)
print(label_counts)
print()
print("Indicies with Yellow label:", label_indicies['Yellow'])

## Oversample Training Set

In [ ]:
# Determine label seen the most
max_count = 0
for label in label_counts:
    max_count = max(max_count, label_counts[label])

# Oversample
oversampled_train_indicies = []
for label in label_indicies:
    random.shuffle(label_indicies[label])
    
    new_label_set = []
    new_label_set += label_indicies[label] * int(max_count / len(label_indicies[label]))
    new_label_set += label_indicies[label][0:max_count % len(label_indicies[label])]
    
    oversampled_train_indicies += new_label_set
    print("Label {} now has {} indicies".format(label, len(new_label_set)))

# Create Generator

In [ ]:
def loadImage(imgPath):
    img = cv2.imread(imgPath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# One Hot Encoder Mapping
label_ohe_map = {
    'NoLight': [1, 0, 0, 0],
    'Red': [0, 1, 0, 0],
    'Yellow': [0, 0, 1, 0], 
    'Green': [0, 0, 0, 1]
}

## Create Generator
def createBatchGenerator(indicies, x, y):
    def batchGenerator(batch_size):
        random.shuffle(indicies)
        
        for batch_i in range(0, len(indicies), batch_size):
            images = []
            labels = []
            
            for index in indicies[batch_i:batch_i + batch_size]:
                image = np.array(loadImage(x[index]))
                image = image / 128 - 1.0 # normalize the image
                label = label_ohe_map[y[index]]
                
                images.append(image)
                labels.append(label)
            
            yield np.array(images), np.array(labels)
        
    return batchGenerator

In [ ]:
trainingBatchGenerator = createBatchGenerator(oversampled_train_indicies, x_train, y_train)
validationBatchGenerator = createBatchGenerator(list(range(len(x_validation))), x_validation, y_validation)
testBatchGenerator = createBatchGenerator(list(range(len(x_test))), x_test, y_test)

## Create Network

In [ ]:
def addConv(input, filters, kernel, stride, padding, keepprob):
    conv = tf.layers.conv2d(input, filters, kernel, stride, padding=padding, \
                            kernel_initializer=tf.truncated_normal_initializer(stddev=0.01), \
                            activation=tf.nn.relu)
    batch_norm = tf.layers.batch_normalization(conv)
    dropout = tf.layers.dropout(batch_norm, keepprob)
    
    return dropout

In [ ]:
# Input Placeholders
images = tf.placeholder(tf.float32, (None, None, None, 3), name='input_images')
images_resized = tf.image.resize_images(images, (600, 800))
labels = tf.placeholder(tf.float32, (None, 4), name='labels')
keepprob = tf.placeholder(tf.float32, name='keep_probability')
learningrate = tf.placeholder(tf.float32, name='learning_rate')

In [ ]:
# Create Network (Custom, Ensemble CNN)

def createEnsembleCNNUnit(input):
    conv1 = addConv(input, 32, 3, 2, 'valid', keepprob)
    max1 = tf.layers.max_pooling2d(conv1, 2, 2, padding='valid')

    conv2 = addConv(max1, 64, 3, 2, 'valid', keepprob)
    max2 = tf.layers.max_pooling2d(conv2, 2, 2, padding='valid')

    conv3 = addConv(max2, 128, 3, 2, 'valid', keepprob)
    max3 = tf.layers.max_pooling2d(conv3, 2, 2, padding='valid')

    conv4 = addConv(max3, 128, 3, 2, 'valid', keepprob)
    max4 = tf.layers.max_pooling2d(conv4, 2, 2, padding='valid')

    flat = tf.layers.flatten(max4)
    fc1 = tf.layers.dense(flat, 128, activation=tf.nn.relu)
    fc2 = tf.layers.dense(fc1, 64, activation=tf.nn.relu)
    logits = tf.layers.dense(fc2, 4, activation=tf.nn.relu)
    
    return logits

unit_1 = createEnsembleCNNUnit(images_resized)
unit_2 = createEnsembleCNNUnit(images_resized)
unit_3 = createEnsembleCNNUnit(images_resized)

logits = tf.add(tf.add(unit_1, unit_2), unit_3, name="logits")
output = tf.nn.softmax(logits, name="output")

In [ ]:
# Create Optimizer
softmax_losses = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
loss = tf.reduce_mean(softmax_losses)
optimizer = tf.train.AdamOptimizer(learningrate).minimize(loss)
correct_prediction = tf.equal(tf.argmax(labels, 1), tf.argmax(output, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Train Network

In [ ]:
# Hyperparameters
epochs = 100
batch_size = 32
lr = 1e-5
kp = 0.7

In [ ]:
# Train Network
sess = tf.Session()

sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

print("Training...")
for e in range(epochs):

    # Train Network
    total_loss = 0
    total_accuracy = 0
    i = 0
    for train_images, train_labels in trainingBatchGenerator(batch_size):
        _, loss_val, acc = sess.run([optimizer, loss, accuracy], {images: train_images, labels: train_labels, \
                                               keepprob: kp, learningrate: lr})
        total_loss += loss_val
        total_accuracy += acc * train_images.shape[0]

        # Print iteration information
        i += 1
        if i % 5 == 0:
            print("   Iteration Loss: {:.5f}, Iteration Accuracy: {:.2f}%".format(loss_val, acc * 100))

    # Determine Validation Accuracy
    validation_loss = 0
    validation_acc = 0
    for valid_images, valid_labels in validationBatchGenerator(batch_size):
        loss_val, acc = sess.run([loss, accuracy], {images: valid_images, labels: valid_labels, keepprob: 1.0})
        validation_loss += loss_val
        validation_acc += acc * valid_images.shape[0]

    print("Epoch {}, Training Loss: {:.5f}, Training Accuracy: {:.2f}%, Validation Loss: {:.5f}, Validation Accuracy: {:.2f}%"\
          .format(e+1, total_loss, total_accuracy * 100 / len(oversampled_train_indicies), \
                 validation_loss, validation_acc * 100 / len(x_validation)))


In [ ]:
# Save Network
saver = tf.train.Saver()
saver.save(sess, './models/cnn_ensemble.ckpt')

## Test Network

In [ ]:
saver = tf.train.Saver()
saver.restore(sess, './models/cnn_ensemble.ckpt')

batch_size = 32

In [ ]:
# Test Accuracy
total_loss = 0
total_acc = 0

for test_images, test_labels in testBatchGenerator(batch_size):
    loss_val, acc = sess.run([loss, accuracy], {images: test_images, labels: test_labels, keepprob: 1.0})
    total_loss += loss_val
    total_acc += acc * test_images.shape[0]

print("Test Loss: {:.5f}, Accuracy: {:.2f}%".format(total_loss, total_acc * 100 / len(x_test)))


## Visually Confirm Working Network

In [ ]:
label_ohe_map_reversed = ['NoLight', 'Red', 'Yellow', 'Green']

In [ ]:
len(x_test)

In [ ]:
desired_test_index = 0

# Load Image
test_img = loadImage(x_test[desired_test_index])
plt.imshow(test_img)

# Run Classification
ohe_index = sess.run(tf.argmax(output, 1), {images: [test_img], keepprob: 1.0})[0]

# Print Information   

print("Network guessed: {}".format(label_ohe_map_reversed[ohe_index]))
print("Actual label: {}".format(y_test[desired_test_index]))
if label_ohe_map_reversed[ohe_index] == y_test[desired_test_index]:
    print("Correct!")
else:
    print("INCORRECT")

## Freeze the Graph into a Protobuf File

In [ ]:
from tensorflow.python.tools.freeze_graph import freeze_graph

In [ ]:
def freeze_graph(model_dir, output_node_names):
    """Extract the sub graph defined by the output nodes and convert 
    all its variables into constant 
    Args:
        model_dir: the root folder containing the checkpoint state file
        output_node_names: a string, containing all the output node's names, 
                            comma separated
    """
    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    if not output_node_names:
        print("You need to supply the name of a node to --output_node_names.")
        return -1

    # We retrieve our checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    # We precise the file fullname of our freezed graph
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_pursuit_model.pb"

    # We clear devices to allow TensorFlow to control on which device it will load operations
    clear_devices = True

    # We start a session using a temporary fresh Graph
    with tf.Session(graph=tf.Graph()) as sess:
        # We import the meta graph in the current default Graph
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

        #[print(n.name) for n in tf.get_default_graph().as_graph_def().node]
        # We restore the weights
        saver.restore(sess, input_checkpoint)

        # We use a built-in TF helper to export variables to constants
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
            output_node_names.split(",") # The output node names are used to select the usefull nodes
        ) 

        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

    return output_graph_def

freeze_graph("./models/", "input_images, logits, labels, output")
